# Look at data
Load LAZ → Clean & Filter → Register Epochs → Compute Height Change → Extract Features → Create ML Dataset


In [29]:
import sys
# !{sys.executable} -m pip install py4dgeo --no-cache-dir 
# !{sys.executable} -m pip install polyscope
# !{sys.executable} -m pip install 'laspy[laszip]'
# !{sys.executable} -m pip install pyproj==3.6.1 geopandas open3d 
# !{sys.executable} -m pip install 'laspy[laszip]'
# !{sys.executable} -m pip install --upgrade pip setuptools wheel
!{sys.executable} -m pip install pdal

  Installing build dependencies ... error
  error: subprocess-exited-with-error
  
  × installing build dependencies for pdal did not run successfully.
  │ exit code: 1
  ╰─> [45 lines of output]
        Using cached scikit_build_core-0.11.6-py3-none-any.whl.metadata (18 kB)
        Using cached numpy-2.3.4.tar.gz (20.6 MB)
        Installing build dependencies: started
        Installing build dependencies: finished with status 'done'
        Getting requirements to build wheel: started
        Getting requirements to build wheel: finished with status 'done'
        Preparing metadata (pyproject.toml): started
        Preparing metadata (pyproject.toml): finished with status 'error'
        error: subprocess-exited-with-error
      
        × Preparing metadata (pyproject.toml) did not run successfully.
        │ exit code: 1
        ╰─> [19 lines of output]
            + /opt/conda/envs/py311/bin/python /tmp/pip-install-h9z_xqlz/numpy_244f631e57524841a793a242fb7203ca/vendored-meson/m

In [5]:
# # import required modules
# import os
# # import py4dgeo
# import numpy as np
# import matplotlib.pyplot as plt
# import matplotlib.dates as mdates
# from datetime import datetime, timedelta

# # add the script assets folder to the path, so that we can import the functions created there
# import sys
# from pathlib import Path
# sys.path.insert(0, str((Path.cwd() / ".." / "utils").resolve()))

# # import point cloud functions
# import pointcloud_functions as pcfuncs

# # specify the data path
# data_dir = '../raw_data/ahk'

# # check if the data path exists
# if not os.path.exists(data_dir):
#     raise Exception('Data path does not exist. Please make sure to specify the correct path.')

# # derive the file paths and read in the data
# pc_file_2020 = f'{data_dir}/ahk_2020_uls.laz'
# pc_file_2021 = f'{data_dir}/ahk_2021_uls.laz'

# # read the point clouds into numpy arrays
# pc_2020 = pcfuncs.read_las(pc_file_2020)
# pc_2021 = pcfuncs.read_las(pc_file_2021)

In [8]:
# pc_2021

array([[6.52793855e+05, 5.18879120e+06, 2.61195270e+03],
       [6.52793855e+05, 5.18879132e+06, 2.61194870e+03],
       [6.52793761e+05, 5.18879146e+06, 2.61185260e+03],
       ...,
       [6.52929120e+05, 5.18943189e+06, 2.39848610e+03],
       [6.52929069e+05, 5.18943141e+06, 2.39876260e+03],
       [6.52929410e+05, 5.18943127e+06, 2.39874820e+03]])

In [7]:
# pc_2020.shape, pc_2021.shape

((37426631, 3), (45662285, 3))

In [24]:
import laspy
import numpy as np
import geopandas as gpd
from shapely.geometry import Point
import open3d as o3d

data_dir = '../raw_data/ahk'
pc_file_2020 = f'{data_dir}/ahk_2020_uls.laz'
pc_file_2021 = f'{data_dir}/ahk_2021_uls.laz'

# Load and Inspect Point Clouds
las2020 = laspy.read(pc_file_2020)
las2021 = laspy.read(pc_file_2021)
print(las2020.header)
print(las2021.header)

# Clip to Stable Parts (Using SHP)
stable_area = gpd.read_file(f"{data_dir}/ahk_2021_uls_stableparts.shp")
stable_area = stable_area.set_crs("EPSG:32632") # is missing in shp package; according to https://3dgeo-heidelberg.github.io/etrainee/data_usecases/usecase_rockglacier_ahk.html

# Filter points within the stable polygon
pts2020 = np.vstack([las2020.x, las2020.y, las2020.z]).T 
# mask = [stable_area.contains(Point(x, y)).any() for x, y in zip(las2020.x, las2020.y)]


# Registration (Align Epochs)
pcd_2020 = o3d.io.read_point_cloud("ahk_2020_uls.laz")
pcd_2021 = o3d.io.read_point_cloud("ahk_2021_uls.laz")

# Perform ICP to align 2020 → 2021
reg = o3d.pipelines.registration.registration_icp(
    pcd_2020, pcd_2021, 0.5, np.eye(4),
    o3d.pipelines.registration.TransformationEstimationPointToPlane()
)
pcd_2020.transform(reg.transformation)


NameError: name 'o3d' is not defined

In [12]:
las2020.header

<LasHeader(1.1, <PointFormat(1, 0 bytes of extra dims)>)>